# Data Mining - What should the next Champion look like?

## Problemdefiniton
In League of Legends beeinflussen neue Champions die Spielbalance. Doch sind bestimmte Champion-Typen unterrepräsentiert? Diese Arbeit nutzt Machine Learning, um vorherzusagen, welche Champion-Eigenschaften im aktuellen Spielgewicht fehlen.

So wird sich die zentrale Frage gestellt: Welche Kombination von Champion-Eigenschaften ist im derzeitigen Spielökosystem am stärksten unterrepräsentiert und könnte als Grundlage für die Entwicklung eines neuen Champions dienen?

## Import - Datenauswahl

### Import notwendiger Bibliotheken


In [ ]:
# Datenverarbeitung & Numerik
import pandas as pd  # Datenverarbeitung
import numpy as np  # Numerische Berechnungen

# Visualisierung
import matplotlib.pyplot as plt  # Basis-Visualisierung
import seaborn as sns  # Erweiterte Visualisierung
%matplotlib inline  # Inline-Darstellung für Jupyter Notebooks

# Datenaufbereitung & Preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder  # Skalierung & Encoding
from sklearn.model_selection import train_test_split  # Aufteilung in Trainings- & Testdaten
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Machine Learning Modelle
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor  # Random Forest Modelle
from sklearn.linear_model import LogisticRegression, LinearRegression  # Lineare Modelle
from sklearn.svm import SVC  # Support Vector Machine Modell
from sklearn.neural_network import MLPRegressor  # Neuronales Netz für Regression
from xgboost import XGBClassifier, XGBRegressor  # Boosted Trees Modelle

# Modellbewertung
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Klassifikationsmetriken
from sklearn.metrics import r2_score, mean_squared_error  # Regressionsmetriken

# Hyperparameter-Tuning
from sklearn.model_selection import GridSearchCV

# Explorative Datenanalyse
from ydata_profiling import ProfileReport  # Automatische EDA

# Sonstiges
import itertools  # Kombinatorik
import pickle  # Modell speichern/laden

### Datenimport

Der folgende Datensatz wurde auf der Website "Kaggle" gefunden, hat seinen Ursprung jedoch in
der Analyse- und Datenplattform MetaSRC. Diese Plattform sammelt, aggregiert und analysiert Match-Daten
um die Trends des Spiels League of Legends darzustellen.
Die Daten werden direkt aus öffentlichen Riot Games-APIs bezogen, wodurch die Statistiken
patch-basiert aktualisiert werden.

In [ ]:
# Datensatz laden
df = pd.read_csv('League of Legends Champion Stats 12.1.csv', sep=';')
df.head()

Der nächste Datensatz entstammt auch Kaggle, hat aber seinen Ursprung in dem "League of Legends Wiki Champion Data Module".
Da dieser Veröffentlichungsdaten beinhaltet, eignet sich dieser für Zeitreihenanalysen
Der Fokus liegt allerdings auf ersterem Datensatz, dieser dient nur als Hilfe für die Zeitreihenanalyse


In [ ]:
df_basic = pd.read_csv('200125_LoL_champion_data.csv')
df_basic.head()

### Merging

In [ ]:
# Verbunden werden die Datensätze durch den gemeinsamen Schlüssel des Champion-Namens
# Dabei wollen wir zum ersten Datensatz die Informationen bezüglich Veröffenlichungsdatum hinzufügen

# Spaltennamen für Konsistenz anpassen
df_basic.rename(columns={'apiname': 'Name'}, inplace=True)

# 'date' Spalte in datetime umwandeln
df_basic['date'] = pd.to_datetime(df_basic['date'], errors='coerce')

# Füge die Veröffentlichungsdaten aus df_basic zum Haupt-Datensatz df hinzu
df = pd.merge(df, df_basic[['Name', 'date']], on='Name', how='left')

df.head()

## EDA und Pre-processing